# Object detection API で学習（くずし字コンペで使ったのをベースにしてる）
- 自宅PC tfgpu_v1-11環境


- faster-RCNN+InceptionResnet


- validation dataなしでクラスは一種類だけにして領域検出だけにする


- 参考
    - https://horomary.hatenablog.com/entry/2018/11/26/214048
    - https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10
    - https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

In [1]:
# importしている別ファイルモジュールを自動でリロード
# https://qiita.com/ishizakiiii/items/b98bbf8997f039f40058
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))# デフォルトは75%

import os, sys
import pandas as pd

# 自作モジュール
sys.path.append(r'C:\Users\shingo\jupyter_notebook\tfgpu_v1-11_work\01_TF_API_tool_module')
from tf_api_tool_module import prepare_anno_csv, generate_tfrecord, prepare_label_map

# 出力先dir
output_dir = 'PATH_TO_BE_CONFIGURED'
os.makedirs(output_dir, exist_ok=True)

# 画像ファイルのパス取得
orig_dir = 'IMAGE_DIR'

# 正解の座標（x1	y1	x2	y2	label_id	file_name）一覧のcsvファイル作成

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

PATH = r'D:\work\kaggle_kuzushiji-recognition\OrigData\kuzushiji-recognition/'
df_train = pd.read_csv(PATH+'train.csv')
unicode_map = {codepoint: char for codepoint, char in pd.read_csv(PATH+'unicode_translation.csv').values}
print ("TRAIN: ", df_train.shape)
#display(df_train.head())

# 276画像にはラベルがありません、dropnaを使ってドロップします
chars = {}
for i in range (df_train.shape[0]):
    try:
        a = [x for x in df_train.labels.values[i].split(' ') if x.startswith('U')]
        n_a = int(len(a))        
        for j in a:
            if j not in chars: chars[j]=1
            else:
                chars[j]+=1
                
        a = " ".join(a)
        
    except AttributeError:
        a = None
        n_a = 0
        
    df_train.loc[i,'chars'] = a
    df_train.loc[i,'n_chars'] = n_a

df_train = df_train[df_train['n_chars']!=0.0]
print ("Drop NoLabel TRAIN: ", df_train.shape)
display(df_train.head())

chars = pd.DataFrame(list(chars.items()), columns=['char', 'count'])
chars['jp_char'] = chars['char'].map(unicode_map)
print (" >> Chars dataframe <<")
print ("Number of chars: ",chars.shape[0])
chars.to_csv("chars_freq.csv",index=False)
display(chars.head())

TRAIN:  (3881, 2)
Drop NoLabel TRAIN:  (3605, 4)


,image_id,labels,chars,n_chars
0,100241706_00004_2,U+306F 1231 3465 133 53 U+304C 275 1652 84 69 ...,U+306F U+304C U+3044 U+3051 U+306B U+306B U+30...,67.0
1,100241706_00005_1,U+306F 1087 2018 103 65 U+304B 1456 1832 40 73...,U+306F U+304B U+304B U+3044 U+306B U+306B U+30...,80.0
2,100241706_00005_2,U+306F 572 1376 125 57 U+306E 1551 2080 69 68 ...,U+306F U+306E U+3078 U+304C U+306B U+3081 U+30...,78.0
3,100241706_00006_1,U+3082 1455 3009 65 44 U+516B 1654 1528 141 75...,U+3082 U+516B U+309E U+306B U+308B U+304B U+30...,72.0
4,100241706_00007_2,U+309D 1201 2949 27 33 U+309D 1196 1539 27 36 ...,U+309D U+309D U+309D U+309D U+3078 U+309D U+25...,167.0


 >> Chars dataframe <<
Number of chars:  4212


,char,count,jp_char
0,U+306F,14759,は
1,U+304C,6740,が
2,U+3044,11903,い
3,U+3051,4224,け
4,U+306B,24685,に


### 画像の枚数が多い+1枚に存在する文字が多い(2-600ぐらい)せいか、欠損なしtarin画像全件(3605枚)だとtfファイルが2日待っても作成できなかった
### n_charaが多いtop100の画像だけをtrain setにしても1枚にすると、数文字しか無い画像の文字は検出できないと思うので、ランダムに取り出した360枚（全画像の1/10）をtrain setにしてみる

In [2]:
import numpy as np
import random

# 乱数固定
np.random.seed(7)
random.seed(7)

rand_index = random.sample(range(df_train.shape[0]), k=df_train.shape[0]//10)
print('rand_index:\n', rand_index)

df_train_rand = df_train.iloc[rand_index]
print(df_train_rand.shape)
display(df_train_rand.head())

df_train = df_train_rand

rand_index:
 [1326, 617, 1617, 2666, 197, 296, 3363, 2194, 385, 1497, 2387, 237, 2078, 879, 153, 352, 1776, 1712, 286, 985, 371, 2257, 1738, 242, 3386, 2316, 507, 914, 2583, 2569, 3594, 253, 2363, 2398, 1624, 203, 905, 190, 2280, 3516, 545, 1186, 1716, 590, 2214, 482, 2338, 1263, 2294, 3342, 2793, 740, 422, 2382, 2339, 2616, 769, 1525, 399, 2243, 2916, 257, 2311, 244, 2535, 843, 2033, 2786, 2177, 1751, 3183, 1286, 1907, 3571, 1856, 1481, 1227, 1017, 3253, 736, 2863, 3194, 999, 335, 2352, 1229, 2151, 2027, 1406, 2987, 1838, 1179, 2494, 299, 483, 2096, 3587, 675, 3101, 1401, 622, 2002, 1727, 160, 2737, 317, 3131, 2285, 2347, 3232, 3351, 1285, 1393, 2847, 1434, 2434, 2034, 2375, 3264, 1868, 281, 3440, 383, 1105, 1941, 2855, 2720, 266, 248, 2994, 2873, 1268, 2650, 2367, 2790, 3366, 1825, 1165, 2935, 1580, 2738, 1421, 92, 1891, 1455, 688, 2502, 479, 2022, 241, 893, 3146, 1177, 529, 3024, 1014, 1629, 1601, 3538, 330, 681, 1839, 1645, 2250, 1138, 560, 3355, 1763, 2253, 1140, 2893, 1701, 1469,

,image_id,labels,chars,n_chars
1501,200015779_00024_2,U+304F 601 2905 27 35 U+3044 516 2763 52 36 U+...,U+304F U+3044 U+30CF U+30F3 U+30F3 U+30FD U+30...,236.0
686,200003076_00078_1,U+306F 798 2934 47 33 U+306F 770 1768 46 34 U+...,U+306F U+306F U+306F U+306F U+306F U+308B U+30...,228.0
1803,200021644_00014_2,U+4E00 1162 1084 89 21 U+4E00 1010 1011 81 25 ...,U+4E00 U+4E00 U+4E00 U+4E00 U+306F U+3078 U+30...,196.0
2897,hnsd001-030,U+30FD 1312 1261 31 33 U+309D 735 1941 37 45 U...,U+30FD U+309D U+30A2 U+30E8 U+306F U+304C U+30...,173.0
209,100249376_00038_1,U+4E03 381 764 69 33 U+5341 380 795 76 36 U+30...,U+4E03 U+5341 U+306F U+4E94 U+306F U+304B U+50...,120.0


# anno_df.tsv 作成
- output_dir で anno_df.tsv の出力先指定する

In [3]:
def conv_df_label_for_one_img(df_series):
    """ df_trainの1行（1画像）のラベル情報をデータフレームに変換する """
    img, labels ,_,_ = df_series
    labels = np.array(labels.split(' ')).reshape(-1, 5)

    file_name = img+'.jpg'
    #print(file_name, labels)

    #col_name = ['Unicode', 'X', 'Y', 'Width', 'Height']# もともとのラベル情報
    #df_label = pd.DataFrame(columns=col_name)
    
    re_col_name = ['x1', 'y1', 'x2', 'y2', 'label_id', 'file_name']# x1	y1	x2	y2	label_id	file_name
    df_re_label = pd.DataFrame(columns=re_col_name)

    for codepoint, x, y, w, h in labels:
        #series = pd.Series([codepoint, x, y, w, h], index=col_name)
        #df_label = df_label.append(series, ignore_index=True)
        
        re_series = pd.Series([x, y, int(x)+int(w), int(y)+int(h), codepoint, file_name], index=re_col_name)
        df_re_label = df_re_label.append(re_series, ignore_index=True)

    #return df_label
    return df_re_label

In [4]:
%%time
df_label_all = None
for index in tqdm(range(df_train.shape[0])): 
    df_re_label = conv_df_label_for_one_img(df_train.values[index])
    #display(df_label)
    #display(df_re_label)
    if df_label_all is None:
        df_label_all = df_re_label
    else:
        df_label_all = df_label_all.append(df_re_label)

print(df_label_all.shape)
print('unique file_name', len(df_label_all['file_name'].unique()))
display(df_label_all.tail())

df_label_all.to_csv(os.path.join(output_dir, 'anno_df.tsv'), sep='\t', index=False)


(66457, 6)
unique file_name 360


,x1,y1,x2,y2,label_id,file_name
191,1050,2281,1133,2410,U+839E,hnsd010-032.jpg
192,499,2209,525,2341,U+3057,hnsd010-032.jpg
193,1041,1731,1104,1864,U+5176,hnsd010-032.jpg
194,1234,2044,1318,2182,U+7B97,hnsd010-032.jpg
195,454,306,533,445,U+5BC4,hnsd010-032.jpg


Wall time: 2min 32s


# Object-Detection-API

## 各ファイルのフォルダパスを指定する
- output_dir で出力先指定する
- anno_df_path で anno_df.tsv の出力先指定する
- xywh_train_csv_path で xywh_train.csv の出力先指定する

In [5]:
# 正解の座標（x1	y1	x2	y2	label_id	file_name）一覧のcsvファイル
anno_df_path = os.path.join(output_dir, 'anno_df.tsv')
xywh_train_csv_path = os.path.join(output_dir, 'xywh_train.csv')
#xywh_valid_csv_path = os.path.join(output_dir, 'xywh_valid.csv')

# 画像ディレクトリ
train_img_dir = os.path.join(orig_dir)
print(train_img_dir)
#valid_img_dir = os.path.join(orig_dir, 'valid_images')

D:\work\kaggle_kuzushiji-recognition\OrigData\kuzushiji-recognition\train_images


## TFrecordの作成
### TFrecord 作るために必要なクラスidとクラスidの対応ファイルclasses_text.csv 作成
- **クラス名は半角英数字しかだめ**

In [6]:
classes_text_df = pd.DataFrame({'id': [1], 'name': [1]})
display(classes_text_df)
# ファイル出力
classes_text_df.to_csv(os.path.join(output_dir, 'classes_text.csv'), sep=',', header=True, index=False)

,id,name
0,1,1


### TFrecord 作るためのアノテーションcsv作成

In [7]:
anno_df = pd.read_csv(anno_df_path, sep='\t')
anno_df.head()

,x1,y1,x2,y2,label_id,file_name
0,601,2905,628,2940,U+304F,200015779_00024_2.jpg
1,516,2763,568,2799,U+3044,200015779_00024_2.jpg
2,1418,1537,1481,1574,U+30CF,200015779_00024_2.jpg
3,1839,2083,1907,2120,U+30F3,200015779_00024_2.jpg
4,1848,2804,1927,2842,U+30F3,200015779_00024_2.jpg


In [8]:
%%time
# anno_dfから正解の座標'filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'）一覧のcsvファイルを作成
# train用
xywh_train_df = prepare_anno_csv.make_xywh_tfAPI_csv(anno_df, train_img_dir, xywh_csv_path=xywh_train_csv_path, is_class_one=True)
# validation用
#xywh_valid_df = prepare_anno_csv.make_xywh_tfAPI_csv(anno_df, valid_img_dir, xywh_csv_path=xywh_valid_csv_path, is_class_one=True)

anno_df
    xmin  ymin  xmax  ymax   class               filename
0   601  2905   628  2940  U+304F  200015779_00024_2.jpg
1   516  2763   568  2799  U+3044  200015779_00024_2.jpg
2  1418  1537  1481  1574  U+30CF  200015779_00024_2.jpg
3  1839  2083  1907  2120  U+30F3  200015779_00024_2.jpg
4  1848  2804  1927  2842  U+30F3  200015779_00024_2.jpg

anno_df.shape (66457, 8)
D:\work\kaggle_kuzushiji-recognition\work\object_detection\20190728\xywh_train.csv
                 filename  width  height class  xmin  ymin  xmax  ymax
0  200015779_00024_2.jpg   2168    3286     1   601  2905   628  2940
1  200015779_00024_2.jpg   2168    3286     1   516  2763   568  2799
2  200015779_00024_2.jpg   2168    3286     1  1418  1537  1481  1574
3  200015779_00024_2.jpg   2168    3286     1  1839  2083  1907  2120
4  200015779_00024_2.jpg   2168    3286     1  1848  2804  1927  2842
Wall time: 13.7 s


### classes_text.csv とアノテーションcsvからgenerate_tfrecord.pyでtfrecordを生成
- generate_tfrecord.py は https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10 から取得


- <font color="Red">%run は毎回カーネル再起動しないと「DuplicateFlagError」（flagsがなぜか二重定義されている）がでる</font>


- **<font color="Red">下記コマンド make_tf_file.bat で実行した</font>**

%%time
```
%run -i C:\Users\shingo\jupyter_notebook\tfgpu_v1-11_work\01_TF_API_tool_module\tf_api_tool_module\generate_tfrecord.py \
    --csv_xywh_id=PATH_TO_BE_CONFIGURED/xywh_train.csv \
    --csv_class_name=PATH_TO_BE_CONFIGURED/classes_text.csv \
    --image_dir=IMAGE_DIR/train_images \
    --output_path=PATH_TO_BE_CONFIGURED/train.record
```

In [5]:
#%%time
#%run -i C:\Users\shingo\jupyter_notebook\tfgpu_v1-11_work\01_TF_API_tool_module\tf_api_tool_module\generate_tfrecord.py \
#    --csv_xywh_id=PATH_TO_BE_CONFIGURED/xywh_valid.csv \
#    --csv_class_name=PATH_TO_BE_CONFIGURED/classes_text.csv \
#    --image_dir=IMAGE_DIR/validation \
#    --output_path=PATH_TO_BE_CONFIGURED/validation.record

## 訓練済みモデルのダウンロード
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md からモデルファイルダウンロード


- D:\work\zoo_model\faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28 に解凍した

## label_map.pbtxtの作成
- label_map.pbtxtはクラス名とidを対応づけるファイル
- **クラス名は半角英数字でないと学習実行時にエラーになる**

In [9]:
classes_text_df = pd.read_csv(os.path.join(output_dir, 'classes_text.csv')) 
label_map_path = os.path.join(output_dir, 'label_map.pbtxt')
prepare_label_map.make_label_map_pbtxt(classes_text_df, label_map_path)

## configファイルの設定
- configファイルのテンプレートはobject_detection/samples/configsに置いてあります。
    - https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs
    - C:\Users\shingo\Git\models\research\object_detection\samples\configs
    
    
- 使用するモデルに対応するテンプレートをダウンロードし、中身を適切に書き換えましょう。


- **訓練済みモデル、TFrecordおよびlable_mp.pbtxtのPATH設定（ PATH_TO_BE_CONFIGURED の箇所を指定するファイルのパスに変更する）と分類クラス数、トレーニングとテストの画像数を変更すればOK**
    - num_classes
    - fine_tune_checkpoint
    - input_path
    - label_map_path
    - num_examples
    
    

- 注意点するべきは以下の点
    - **フルパスで記述すること**
    - **￥ではなく/を使うこと（winでも）**
    - **パスは単一引用符（ '）ではなく、二重引用符（ "）で囲む必要がある**
    - **「fine_tune_checkpoint:」 のパスは /model.ckpt までにすること**
    

- **<font color="Red">下記コマンド sed_config_file.bat で実行した</font>**

```
%run -i C:\Users\shingo\jupyter_notebook\tfgpu_v1-11_work\01_TF_API_tool_module\tf_api_tool_module\prepare_coco_config.py \
        --config_path C:\Users\shingo\jupyter_notebook\tfgpu_v1-11_work\01_TF_API_tool_module\tf_api_train_base_files\faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco_edit.config \
        --out_path PATH_TO_BE_CONFIGURED/faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco_edit_sed.config \
        --train_record_path PATH_TO_BE_CONFIGURED/train.record \
        --val_record_path PATH_TO_BE_CONFIGURED/validation.record \
        --label_map_pbtxt_path PATH_TO_BE_CONFIGURED/label_map.pbtxt \
        --eval_num_examples 0 \
        --num_classes 1 \
        --max_detections_per_class 650 \
        --max_total_detections 650 \
        --first_stage_max_number_of_boxes 650 
```                            

### configファイルの追加設定
- 参考：追加設定入れたconfigファイル
    - D:\work\JT_SIGNATE_Contest\02_object_detection\result\object_detection_API_v4
      faster_rcnn_inception_resnet_v2_atrous_coco_sed_v4_all_train.config


#### メモリに関連するパラメータ（メモリ足りない時の処置）
- **<font color="Red">batch_queue_capacity, prefetch_queue_capacity, queue_capacity, min_after_dequeue　を追加する</font>**
- 参考：
    - https://github.com/anand-sonawane/object-detection-tf-api
    - https://towardsdatascience.com/creating-your-own-kotlin-detector-in-tensorflow-a425efcdc68b
    - https://stackoverflow.com/questions/45247292/ram-error-running-the-tensorflow-object-detection-api
    - https://github.com/tensorflow/models/issues/3408
    
    例
    ```
    train_config: {
      ... other settings  
      gradient_clipping_by_norm: 10.0
      batch_queue_capacity: 2
      prefetch_queue_capacity: 2
    }

    train_input_reader: {
      tf_record_input_reader {
        input_path: "/path/to/train.tfrecord"
      }
      label_map_path: "/path/to/label/map.pbtxt"
      queue_capacity: 100
      min_after_dequeue: 50
    }

    eval_input_reader: {
      tf_record_input_reader {
        input_path: "/path/to/eval.tfrecord"
      }
      label_map_path: "/path/to/label/map.pbtxt"
      shuffle: true
      queue_capacity: 20
      min_after_dequeue: 10
      num_readers: 1
    }
    ```


#### **<font color="Red">画像水増しする場合は 以下のようにdata_augmentation_options に追加する</font>**
- 参考: 
    - https://devtalk.nvidia.com/default/topic/1044680/tensorrt/run-ssd_mobilenetv2-tensorflow-object-detection-api-on-tensorrt/
    - https://pasticciodiopinioni.wordpress.com/2017/09/12/data-augmentation-options-in-tensorflow-object-recognition/  
    - https://github.com/tensorflow/models/issues/5445
        
    例  
    ```
  data_augmentation_options {
    random_adjust_brightness {
      max_delta: 0.10000000149
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
      min_delta: 0.899999976158
      max_delta: 1.10000002384
    }
  }
  data_augmentation_options {
    random_adjust_hue {
      max_delta: 0.0299999993294
    }
  }
  data_augmentation_options {
    random_adjust_saturation {
      min_delta: 0.949999988079
      max_delta: 1.04999995232
    }
  }
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_vertical_flip {
    }
  }
  data_augmentation_options {
    random_rgb_to_gray {
      probability : 0.3
    }
  }
  data_augmentation_options {
    random_image_scale {
      min_scale_ratio: 0.3
      max_scale_ratio: 1.5
    }
  }
    ```

## モデルのトレーニング
- object_detection\utils\learning_schedules.py の168行目のコード変更必要だった
    - list(range(num_boundaries)),#range(num_boundaries), # https://github.com/tensorflow/models/issues/3705#issuecomment-375563179 より
    
    
- jupyterからは実行できなかった（できるはずだが標準出力が膨大に出るのでコマンドラインから実行すべき）


- **引数のパスは「\」を使うとエラー。「/」区切りでないと実行できない！！**


- --train_dir で指定したディレクトリにチェックポイントファイルが保存される


- **<font color="Red">下記コマンド run_train.bat で実行した</font>**


```
python C:/Users/shingo/Git/models/research/object_detection/train.py 
--logtostderr 
--train_dir=PATH_TO_BE_CONFIGURED/log_train 
--pipeline_config_path=PATH_TO_BE_CONFIGURED/faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco_edit_sed.config 
```

## TensorBoard起動してログ確認
- コマンドラインで 
```
tensorboard --logdir=D:/work/kaggle_kuzushiji-recognition/work/object_detection/20190728/log_train
```
- http://localhost:6006 にTensorBoard表示される


- **<font color="Red">上記コマンド run_tensorboard.bat で実行した</font>**


- モデル評価は以下のコマンドだがCOCO APIが無いから実行できないみたい
    - https://segmentfault.com/a/1190000018080669 より

```
python C:/Users/shingo/Git/models/research/object_detection/eval.py --logtostderr --pipeline_config_path=PATH_TO_BE_CONFIGURED/faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco_edit_sed.config --checkpoint_dir=PATH_TO_BE_CONFIGURED/log_train/ --eval_dir=PATH_TO_BE_CONFIGURED/eval
```

## トレーニングが終わったらinference_graphを出力(model.ckpt-以下の数字は例）
- D:\work\kaggle_kuzushiji-recognition\work\object_detection\20190728/inference_graph にモデル作成する


- **<font color="Red">下記コマンド deploy_trained_model.bat で実行できる</font>**

```
%run -i C:\Users\shingo\Git\models\research\object_detection\export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path PATH_TO_BE_CONFIGURED/faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco_edit_sed.config \
    --trained_checkpoint_prefix PATH_TO_BE_CONFIGURED/log_train/model.ckpt-196725 \
    --output_directory PATH_TO_BE_CONFIGURED/inference_graph
```